In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [2]:
cur_reward = 0

In [3]:
import math

import cv2
import numpy as np
import random

step = 1

canvas = np.zeros((1000, 1000, 3), np.uint8)
target_pos = [[300, 300], [400, 400]]
car_pos = 0


def car_init():
    global car_pos, target_pos
    x_pos_rand = random.randint(0, 900)
    y_pos_rand = random.randint(0, 900)
    x1_y1 = (x_pos_rand, y_pos_rand)
    x2_y2 = (x_pos_rand + 100, y_pos_rand + 100)
    car_pos = (x1_y1, x2_y2)
    x_pos_rand = random.randint(0, 900)
    y_pos_rand = random.randint(0, 900)
    x1_y1 = (x_pos_rand, y_pos_rand)
    x2_y2 = (x_pos_rand + 100, y_pos_rand + 100)
    target_pos = (x1_y1, x2_y2)
    _,car_center,target_center = get_car_status(car_pos, target_pos)
    return car_center, target_center


def get_car_status(cp, tg):
    center_car = (int((cp[0][0] + cp[1][0]) / 2), int((cp[0][1] + cp[1][1]) / 2))
    center_target = (int((tg[0][0] + tg[1][0]) / 2), int((tg[0][1] + tg[1][1]) / 2))
    distance = int(math.sqrt((center_target[0] - center_car[0])**2 + (center_target[1] - center_car[1])**2))

    return distance, center_car, center_target

def get_car_distance():
    d, _, _ = get_car_status(car_pos, target_pos)
    return d

def car_mov(msg):
#     print(msg)
    global car_pos
    global canvas, cur_reward

    x_y_adjust = [0, 0]
    
    if msg == 0:  # forward
        x_y_adjust = 0, step
    elif msg == 1:  # backward
        x_y_adjust = 0, -step
    elif msg == 2:  # right forward
        x_y_adjust = step, step
    elif msg == 3:  # left forward
        x_y_adjust = -step, step
    elif msg == 4:  # right backward
        x_y_adjust = step, -step
    elif msg == 5:  # left backward
        x_y_adjust = -step, -step
    elif msg == 6:  # stop
        x_y_adjust = 0, 0

    updated_x_y_1 = car_pos[0][0] + x_y_adjust[0], car_pos[0][1] + x_y_adjust[1]
    updated_x_y_2 = car_pos[1][0] + x_y_adjust[0], car_pos[1][1] + x_y_adjust[1]

    car_pos = (updated_x_y_1, updated_x_y_2)

    canvas = np.zeros((1000, 1000, 3), np.uint8)
    canvas = cv2.rectangle(canvas, target_pos[0], target_pos[1], (0, 0, 255), -1)
    canvas = cv2.rectangle(canvas, car_pos[0], car_pos[1], (0, 255, 0))

    distance, car_center, target_center = get_car_status(car_pos, target_pos)

    cv2.circle(canvas, car_center, 10, (0, 255, 255), -1)
    cv2.circle(canvas, target_center, 10, (0, 255, 255), -1)
    cv2.line(canvas, car_center, target_center, (255, 255, 0), thickness=5)
    cv2.putText(canvas, 'dis: ' + str(distance), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 50), 2, cv2.LINE_AA)
    cv2.putText(canvas, 'reward: ' + str(cur_reward), (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 50), 2, cv2.LINE_AA)
    cv2.imshow('img', canvas)

    cv2.waitKey(1)

    _,car_center,target_center = get_car_status(car_pos, target_pos)
    return car_center, target_center  


# car_init()
# car_mov(2)
# car_mov(2)
# car_mov(2)
# cv2.destroyAllWindows()

In [8]:
def ard_write(x):
    arduino.write(bytes(str(x), 'utf-8'))

def ard_read():
    ard_write('s')
    data = arduino.readline()[:-2]
    return int(data.decode())

In [9]:
del arduino

NameError: name 'arduino' is not defined

In [4]:
# from time import sleep
# import numpy as np

# reward_step = 5
# global cur_reward
# class Crazy_Taxi(Env):
#     def __init__(self):
#         global cur_reward
#         self.action_space = Discrete(7)
#         # f, b, fr, fl, br, bl, Stop
#         # 1, 2, 3, 4, 5, 6, 7
#         low = np.zeros(4)
#         high = np.ones(4) * 1000
#         self.observation_space = Box(low=low, high=high, dtype=int)
#         self.statetmp = car_init() # return xc, yc, xt, yt
#         (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
#         self.state = self.xc, self.yc, self.xt, self.yt
#         self.distance = get_car_distance()
#         self.last_distance = self.distance
#         self.episode_length = 100
#         cur_reward = 0

                
#     def step(self, action):
#         global cur_reward

# #         print(action)
#         self.episode_length -= 1
# #         self.state = car_mov(action)
        
#         self.statetmp = car_mov(action) # return xc, yc, xt, yt
#         (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
#         self.state = self.xc, self.yc, self.xt, self.yt
        
#         self.distance = get_car_distance()
#         if(self.distance == 0):
#             reward = reward_step
            
#         elif(self.distance < self.last_distance):
#             reward = reward_step 
#         else:
#             reward = -reward_step
            
#         cur_reward+=reward
        
#         if self.episode_length == 0 :
#             done = True
#         else:
#             done = False
        
#         self.last_distance = self.distance
        
#         return self.state, reward, done, {}

#     def render(self):
#         # Implement viz
#         pass
    
#     def reset(self):
#         global cur_reward
#         self.episode_length = 100
#         self.statetmp = car_init() # return xc, yc, xt, yt
#         (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
#         self.state = self.xc, self.yc, self.xt, self.yt
#         self.distance = get_car_distance()
#         self.last_distance = self.distance
#         cur_reward = 0
#         return self.state
    

In [4]:
from time import sleep
import numpy as np

reward_step = 1
l = 1
global cur_reward
class Crazy_Taxi(Env):
    def __init__(self):
        global cur_reward, l
        self.action_space = Discrete(7)
        # f, b, fr, fl, br, bl, Stop
        # 1, 2, 3, 4, 5, 6, 7
        low = np.zeros(2) * -1
        high = np.ones(2) * 1
        self.observation_space = Box(low=low, high=high, dtype=int)
        self.statetmp = car_init() # return xc, yc, xt, yt
        (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
        self.arr = np.array([self.xc, self.yc, self.xt, self.yt], dtype=float)
        self.norm = np.linalg.norm(self.arr)
        self.na = self.arr/self.norm
#         self.state = self.xc, self.yc, self.xt, self.yt
        self.state = self.na[0],self.na[1],self.na[2],self.na[3]
        self.distance = get_car_distance()
        self.last_distance = self.distance
        self.last_state = self.state
#         print("is",self.state)
#         print("il",self.last_state)
        self.state_v2 = self.na[0]-self.na[2], self.na[1]-self.na[3]
        self.episode_length = l
        cur_reward = 0

                
    def step(self, action):
        global cur_reward
        reward = 0
#         print(action)
        self.episode_length -= 1
#         self.state = car_mov(action)
        
        self.statetmp = car_mov(action) # return xc, yc, xt, yt
        (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
        
        self.arr = np.array([self.xc, self.yc, self.xt, self.yt], dtype=float)
        self.norm = np.linalg.norm(self.arr)
        self.na = self.arr/self.norm
#         self.state = self.xc, self.yc, self.xt, self.yt
        self.state = self.na[0],self.na[1],self.na[2],self.na[3]
        self.state_v2 = self.na[0]-self.na[2], self.na[1]-self.na[3]
               
        
        lxc, lyc, lxt, lyt = self.last_state
#         print("s",self.state)
#         print("l",self.last_state)
        self.distance = get_car_distance()
        reward = self.last_distance - self.distance
        if(reward == 0 and self.distance <= step):
            reward += 10
        cur_reward+=reward
        
        if self.episode_length == 0 :
            done = True
        else:
            done = False
        
        self.last_distance = self.distance
        self.last_state = self.state
        
        return self.state_v2, reward, done, {}

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        global cur_reward, l
        self.episode_length = l
        self.statetmp = car_init() # return xc, yc, xt, yt
        (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
        
        self.arr = np.array([self.xc, self.yc, self.xt, self.yt], dtype=float)
        self.norm = np.linalg.norm(self.arr)
        self.na = self.arr/self.norm
#         self.state = self.xc, self.yc, self.xt, self.yt
        self.state = self.na[0],self.na[1],self.na[2],self.na[3]
        self.distance = get_car_distance()
        self.last_distance = self.distance
        self.last_state = self.state
        self.distance = get_car_distance()
        self.last_distance = self.distance
        self.state_v2 = self.na[0]-self.na[2], self.na[1]-self.na[3]
        cur_reward = 0
        return self.state_v2
    

In [65]:
# from time import sleep
# import numpy as np

# reward_step = 5
# global cur_reward
# class Crazy_Taxi(Env):
#     def __init__(self):
#         global cur_reward
#         self.action_space = Discrete(7)
#         # f, b, fr, fl, br, bl, Stop
#         # 1, 2, 3, 4, 5, 6, 7
#         low = np.zeros(4)
#         high = np.ones(4) * 1000
#         self.observation_space = Box(low=low, high=high, dtype=int)
#         self.statetmp = car_init() # return xc, yc, xt, yt
#         (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
#         self.state = self.xc, self.yc, self.xt, self.yt
#         self.distance = get_car_distance()
#         self.last_distance = self.distance
#         self.episode_length = 1
#         cur_reward = 0

                
#     def step(self, action):
#         global cur_reward

# #         print(action)
#         self.episode_length -= 1
# #         self.state = car_mov(action)
        
#         self.statetmp = car_mov(action) # return xc, yc, xt, yt
#         (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
#         self.state = self.xc, self.yc, self.xt, self.yt
        
#         self.distance = get_car_distance()
#         if(self.distance == 0):
#             reward = reward_step
            
#         elif(self.distance < self.last_distance):
#             reward = reward_step 
#         else:
#             reward = -reward_step
            
#         cur_reward+=reward
        
#         if self.episode_length == 0 :
#             done = True
#         else:
#             done = False
        
#         self.last_distance = self.distance
        
#         return self.state, reward, done, {}

#     def render(self):
#         # Implement viz
#         pass
    
#     def reset(self):
#         global cur_reward
#         self.episode_length = 1
#         self.statetmp = car_init() # return xc, yc, xt, yt
#         (self.xc, self.yc), (self.xt, self.yt) = self.statetmp
#         self.state = self.xc, self.yc, self.xt, self.yt
#         self.distance = get_car_distance()
#         self.last_distance = self.distance
#         cur_reward = 0
#         return self.state
    

In [5]:
env = Crazy_Taxi()

In [6]:
env.reset()

(0.5087070304212464, 0.5458555318976117)

In [7]:
env.episode_length

1

In [8]:
env.distance

984

In [9]:
env.observation_space.sample().shape

(2,)

In [10]:
env.state

(0.6792868841392501,
 0.6974820685358372,
 0.17057985371800366,
 0.15162653663822548)

In [11]:
env.state_v2

(0.5087070304212464, 0.5458555318976117)

In [31]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D
import keras
from keras.optimizers import adam_v2
import tensorflow as tf

In [32]:
states = env.observation_space.shape
actions = env.action_space.n

In [33]:
states

(2,)

In [34]:
actions

7

In [35]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render() ## lol sha8al ! :)))))))))
        action = env.action_space.sample()
        print(action)
        n_state, reward, done, info = env.step(action)
        print(n_state, reward, done, info)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

2
(-0.09652740733395049, -0.049522756806113705) 1 True {}
Episode:1 Score:1
1
(0.5716726737274942, 0.05104220301138346) 0 True {}
Episode:2 Score:0
5
(0.43285961951232127, 0.17461428387970496) 2 True {}
Episode:3 Score:2
6
(0.3894316468388741, 0.38108668297804105) 0 True {}
Episode:4 Score:0
6
(0.28949976275792094, 0.5461421904788999) 0 True {}
Episode:5 Score:0
6
(0.14019197051836535, 0.11432321405366708) 0 True {}
Episode:6 Score:0
0
(0.4523526934047183, -0.49574383905505104) 1 True {}
Episode:7 Score:1
6
(-0.10004566325796499, 0.04081863060924967) 0 True {}
Episode:8 Score:0
2
(0.2197669932890336, -0.30415751871202246) 1 True {}
Episode:9 Score:1
2
(0.34022332795575067, 0.2965030404848014) -1 True {}
Episode:10 Score:-1


In [36]:
keras.backend.clear_session()

In [37]:
states

(2,)

In [38]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='sigmoid', input_shape=(1,2)))
    model.add(Dense(24, activation='sigmoid'))
    model.add(Dense(10, activation='sigmoid'))
    model.add(Flatten())
#     model.add(Dense(7, activation='softmax'))
    model.add(Dense(actions, activation='linear'))
    return model

In [39]:
del model

NameError: name 'model' is not defined

In [40]:
model = build_model(states, actions)

In [41]:
model.input_shape

(None, 1, 2)

In [42]:
model.output_shape

(None, 7)

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 24)             72        
                                                                 
 dense_1 (Dense)             (None, 1, 24)             600       
                                                                 
 dense_2 (Dense)             (None, 1, 10)             250       
                                                                 
 flatten (Flatten)           (None, 10)                0         
                                                                 
 dense_3 (Dense)             (None, 7)                 77        
                                                                 
Total params: 999
Trainable params: 999
Non-trainable params: 0
_________________________________________________________________


In [44]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [45]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(adam_v2.Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)
cv2.destroyAllWindows()

Training for 100000 steps ...
Interval 1 (0 steps performed)


C:\Users\Princ\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   11/10000 [..............................] - ETA: 1:47 - reward: 0.1818  

C:\Users\Princ\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 9364/10000 [===========================>..] - ETA: 23s - reward: 0.5587

In [172]:
# dqn = build_agent(model, actions)
# dqn.compile(adam_v2.Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)
cv2.destroyAllWindows()

Training for 50000 steps ...
Interval 1 (0 steps performed)
 3395/10000 [=========>....................] - ETA: 2:18 - reward: 1.2837done, took 74.303 seconds


In [36]:
cv2.destroyAllWindows()

In [87]:
dqn.save_weights("best2.h5f")

In [88]:
del model
del dqn

In [52]:
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(adam_v2.Adam(lr=1e-3), metrics=['mae'])

In [53]:
dqn.load_weights('best.h5f')

In [183]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)
cv2.destroyAllWindows()

Training for 10000 steps ...
Interval 1 (0 steps performed)
  213/10000 [..............................] - ETA: 4:48 - reward: 6.1408done, took 6.327 seconds


In [ ]:
dqn.test(env, nb_episodes=100, visualize=False)
cv2.destroyAllWindows()

Testing for 100 episodes ...
Episode 1: reward: 272.000, steps: 500
Episode 2: reward: 263.000, steps: 500
Episode 3: reward: 624.000, steps: 500


In [214]:
model.save("x.h5")

In [101]:
model = keras.models.load_model("24-10.h5")

In [ ]:
int(ard_read().decode())

In [35]:
len(data_shape)

NameError: name 'data_shape' is not defined

In [36]:
list(model.output.shape)

[None, 8]

In [ ]:
list((None, actions))

In [ ]:
ard_write('0')